Import Packages

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor
import optuna.integration.lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings("ignore")

（Load Data）

In [ ]:
prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv", parse_dates=["Date"])

# EDA

In [ ]:
prices

column information<br>
(1) RowId: ID Date_SecuritiesCode, Unique ID of price records <br>
(2) Date: Trade date <br>
(3) SecuritiesCode: Local securities code, 2000 <br>
(4) Open: First traded price on a day<br>
(5) High: Highest traded price on a day <br>
(6) Low : Lowest traded price on a day <br>
(7) Close: Last traded price on a day <br>
(8) Volume: Number of traded stocks on a day <br>
(9) AdjustmentFactor：Change in stock price due to a split/reverse split. Correction from the closing price to the beginning of the next day.）<br>
(10) ExpectedDividend: Projected dividend amount <br>
(11) SupervisionFlag: Flag for supervised and delisted stocks. High risk of delisting. <br>
(12) Target: Target variable; rate of return derived from the difference between one and two days later <br>

## Check missing values

In [ ]:
#Check the number of missing values per column
prices.isnull().sum()

### 

In [ ]:
#"Open", "High", "Low", "Close"（（Check about missing values of "Open", "High", "Low", "Close"）
print((prices["Open"].isnull() == (prices["Volume"]==0)).all())
print((prices["High"].isnull() == (prices["Volume"]==0)).all())
print((prices["Low"].isnull() == (prices["Volume"]==0)).all())
print((prices["Close"].isnull() == (prices["Volume"]==0)).all())
#"Volume"（"Volume" represents the total amount traded on that day.）
#（So, "Volume = 0" means that there was no trading on that day, and the opening, high, low, and closing prices are null.）

In [ ]:
#"ExpectedDividend"（Check about missing values of ""ExpectedDividend"）
prices[(~prices["ExpectedDividend"].isnull())]["Date"].value_counts()
#（Value exists only on certain days. The value may be filled at the time of closing.）

In [ ]:
#（Check for missing dates）
prices["SecuritiesCode"].value_counts().sort_values()
#（1413 misses 21days, 8806 misses 20days, 4699 misses 1day）

In [ ]:
#（Check for missing dates）
pd.pivot(prices, index="Date", columns="SecuritiesCode", values="Volume")[[1413, 8806, 4699, 1375]].tail(30)
#（1413 misses 2022-04-25～2022-05-27）
#（8806 misses 2022-04-26～2022-05-27）
#（4699 misses 2022-05-27）

Feature generation and selection）

In [ ]:
def MA(series, window=25):
    return series.rolling(window, min_periods=1).mean()

def DMA(series, window=25):
    return series/MA(series, window) - 1

def divergence(series, window=25):
    std = series.rolling(window,min_periods=1).std()
    mean = series.rolling(window,min_periods=1).mean()
    return (series-mean) / std    

def rsi(series, n=14):
    return (series - series.shift(1)).rolling(n).apply(lambda s:s[s>0].sum()/abs(s).sum())

def stochastic(series, k=14, n=3, m=3):
    _min = series.rolling(k).min()
    _max = series.rolling(k).max()
    _k = (series - _min)/(_max - _min)
    _d1 = _k.rolling(n).mean()
    _d2 = _d1.rolling(m).mean()
    return pd.DataFrame({
                    "%K":_k,
                    "FAST-%D":_d1,
                    "SLOW-%D":_d2,
                    },index=series.index)
    # return _k, _d1, _d2

def psy(series, n=14):
    return (series - series.shift(1)).rolling(n).apply(lambda s:(s>=0).mean())

def ICH(series):
    conv = series.rolling(9).apply(lambda s:(s.max()+s.min())/2)
    base = series.rolling(26).apply(lambda s:(s.max()+s.min())/2)
    pre1 = ((conv + base)/2).shift(25)
    pre2 = d.Close_adj.rolling(52).apply(lambda s:(s.max()+s.min())/2).shift(25)
    lagg = d.Close_adj.shift(25)
    return conv, base, pre1, pre2, lagg

def roc(series, window=14):
    return series/series.shift(window) - 1

class FeatureBase():
    def create_feature(self, d):
        assert False, "NotImplemented"
        
class MAFeature(FeatureBase):
    def create_feature(self, d):
        return self._create_feature(d["Close_adj"])

    def _create_feature(self, series, window1=5, window2=25):
        ma1 = MA(series, window1).rename("MA1")
        ma2 = MA(series, window2).rename("MA2")
        diff = ma1 - ma2
        cross = pd.Series(
                        np.where((diff>0) & (diff<0).shift().fillna(False), 1,
                            np.where((diff<0) & (diff>0).shift().fillna(False), -1, 0
                                )
                        ),
                        index = series.index, name="MA_Cross"
                )
        return pd.concat([ma1, ma2, cross], axis=1)

In [ ]:
def holiday(d):
    return pd.DataFrame({
        "before_holiday":(d["Date"] != d["Date"].shift(-1) - datetime.timedelta(days=1)) | (d["weekday"]==4),
        "after_holiday":(d["Date"] != d["Date"].shift(1) + datetime.timedelta(days=1)) | (d["weekday"]==0)
    }, index=d.index)
def make_features(df):
    df = df[[
        "Date","SecuritiesCode","Open","Close","AdjustmentFactor",
        "Volume"
    ]].copy()
    df["weekday"] = df["Date"].dt.weekday
    df = df.join(df.groupby("SecuritiesCode").apply(holiday))
    df["Volume_ratio"] = df["Volume"]/df.groupby("SecuritiesCode")["Volume"].rolling(window=15, min_periods=1).mean().reset_index("SecuritiesCode",drop=True)
    df["Close_adj"] = df.groupby("SecuritiesCode").apply(lambda d:d["Close"]/d["AdjustmentFactor"].cumprod().shift().fillna(1)).reset_index("SecuritiesCode",drop=True)
    df[["MA1", "MA2", "MA_Cross"]] = df.groupby("SecuritiesCode").apply(lambda d: MAFeature()._create_feature(d.Close_adj))# .join(df["Target"].shift(-1)).groupby("MA_Cross").describe()
    df["Diff"] = (df["Close"] - df["Open"]) / df[["Close","Open"]].mean(axis=1)
    df["Diff_MA1"] = df["Close_adj"] - df["MA1"]
    df["Diff_MA2"] = df["Close_adj"] - df["MA2"]
    for i in range(1, 3):
        df["MA_Cross_lag_{:}".format(i)] = df.groupby("SecuritiesCode")["MA_Cross"].shift(i)

    df["DivMA"] = df.groupby("SecuritiesCode")["Close_adj"].apply(DMA)
    df["Div"] = df.groupby("SecuritiesCode")["Close_adj"].apply(divergence)
    df["Rsi"] = df.groupby("SecuritiesCode")["Close_adj"].apply(rsi)
    df = df.join(df.groupby("SecuritiesCode")["Close_adj"].apply(stochastic))
    
    ##################（Add features）#######################
    
    ##################（Add features）#######################
    return df

In [ ]:
columns = [
    "Diff", "Close_adj","Volume_ratio",
    "before_holiday", "after_holiday",
    "Diff_MA1", "Diff_MA2","MA_Cross",
    'MA_Cross_lag_1', 'MA_Cross_lag_2',
    "DivMA", "Div", "Rsi", "%K", "FAST-%D","SLOW-%D",
]

# （Build model）

In [ ]:
%%time
df = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv", parse_dates=["Date"])
df = make_features(df).join(df.Target)

In [ ]:
def train_model(X, y):
#     fast
    model=LGBMRegressor(boosting_type="dart",
                        num_leaves=31,max_depth=12,
                        learning_rate=0.2, n_estimators=100,
                        random_state=0)
#     model=LGBMRegressor(boosting_type="dart",
#                     num_leaves=31,max_depth=12,
#                     learning_rate=0.02, n_estimators=1000,
#                     random_state=0)
    model.fit(X,y)
    # model.score(X,y)
    return model

In [ ]:
# %%time
# def train_model_val(X, y, validation=True):
#     model=LGBMRegressor(boosting_type="dart",
#                         num_leaves=31,max_depth=12,
#                         learning_rate=0.1, n_estimators=1000,
#                         random_state=42)

#     if validation:
#         valid_ratio = 0.2
#         n = int(len(X) * valid_ratio)
#         model.fit(X.iloc[:-n].values, y.iloc[:-n].values)
#         score = model.score(X.iloc[n:],y.iloc[n:])
#     else:
#         score = np.nan
    
#     model.fit(X.values,y.values)
#     return model, score

# columns2 = columns + list(transformed.columns)
# models2 = {}
# scores = {}
# for code, d in df.groupby("SecuritiesCode"):
#     _d = d[~d.Target.isnull()].set_index("Date")
#     X = _d[columns].join(transformed)[columns2]
#     y = _d.Target
#     model, score = train_model2(X, y)
#     models2[code] = model
#     scores[code] = score
#     print("Validation",code, score)

In [ ]:
%%time
models = {}
num = 0
for code, d in df.groupby("SecuritiesCode"):
    d = d[~d.Target.isnull()]
    X = d[columns]
    y = d.Target
    model = train_model(X, y)
    models[code] = model

# (evaluation)

In [ ]:
import warnings, gc
import numpy as np 
import pandas as pd
import matplotlib.colors
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
# from datetime import datetime, timedelta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error,mean_absolute_error
from lightgbm import LGBMRegressor
from decimal import ROUND_HALF_UP, Decimal
warnings.filterwarnings("ignore")
import plotly.figure_factory as ff

In [ ]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
#         weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        df_len = len(df)
        if df_len>=portfolio_size:
            weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
            purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        else:
            weights = np.linspace(start=toprank_weight_ratio, stop=1, num=df_len)
            purchase = (df.sort_values(by='Rank')['Target'][:df_len] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

In [ ]:
%%time
ts_fold = TimeSeriesSplit(n_splits=5, gap=0)
df_input = df.copy().sort_values(['Date','SecuritiesCode'])
# prices=price_features.dropna().sort_values(['Date','SecuritiesCode'])
y=df_input['Target'].to_numpy()
X=df_input.drop(['Target'],axis=1)

feat_importance=pd.DataFrame()
sharpe_ratio=[]
result = {}
    
for fold, (train_idx, val_idx) in enumerate(ts_fold.split(X, y)):
    
    print("\n========================== Fold {} ==========================".format(fold+1))
    df_train = df_input.iloc[train_idx,:]
    df_valid = df_input.iloc[val_idx,:]
    X_train, y_train = X.iloc[train_idx,:], y[train_idx]
    X_valid, y_val = X.iloc[val_idx,:], y[val_idx]
    
    print("Train Date range: {} to {}".format(X_train.Date.min(),X_train.Date.max()))
    print("Valid Date range: {} to {}".format(X_valid.Date.min(),X_valid.Date.max()))
    
#     X_train.drop(['Date','SecuritiesCode'], axis=1, inplace=True)
#     X_train.drop(['Date'], axis=1, inplace=True)
#     X_val=X_valid[X_valid.columns[~X_valid.columns.isin(['Date','SecuritiesCode'])]]
#     val_dates=X_valid.Date.unique()[1:-1]
#     print("\nTrain Shape: {} {}, Valid Shape: {} {}".format(X_train.shape, y_train.shape, X_val.shape, y_val.shape))
    
#     params = {'n_estimators': 1000,
#               'num_leaves' : 31,
#               'max_depth' : 12,
#               'learning_rate': 0.02,
#               'colsample_bytree': 0.9,
#               'subsample': 0.8,
#               'reg_alpha': 0.4,
#               'metric': 'mae',
#               'random_state': 0,
#               'boosting_type': "dart"
#               }

#     gbm = LGBMRegressor(**params).fit(X_train, y_train, 
#                                       eval_set=[(X_train, y_train), (X_val, y_val)],
#                                       verbose=300, 
#                                       eval_metric=['mae','mse'])

    models = {}
    for code, d in df_train.groupby("SecuritiesCode"):
        X_tmp = d[columns]
        y_tmp = d.Target
        model = train_model(X_tmp[~y_tmp.isnull()], y_tmp[~y_tmp.isnull()])
        models[code] = model
        
#     gbm = train_model(X_train, y_train)
#     y_pred = gbm.predict(X_val)
#     rmse = np.sqrt(mean_squared_error(y_val, y_pred))
#     mae = mean_absolute_error(y_val, y_pred)
#     feat_importance["Importance_Fold"+str(fold)]=gbm.feature_importances_
#     feat_importance.set_index(X_train.columns, inplace=True)
    
    rank=[]
    df_valid_tmp = df_valid.copy()
#     X_val_df=X_valid[X_valid.Date.isin(val_dates)]
    for i, d in df_valid_tmp.groupby("SecuritiesCode"):
        df_valid_tmp.loc[d.index, "pred"]=models[i].predict(d[columns])
    for date, d in df_valid_tmp.groupby("Date"):
        df_valid_tmp.loc[d.index, "Rank"]= (d.pred.rank(method="first", ascending=False)-1).astype(int)
#     val_df_tmp = X_val_df.merge(y_val)
    sharpe=calc_spread_return_sharpe(df_valid_tmp)
    result[fold] =  df_valid_tmp
    
#     for i in X_val_df.Date.unique():
#         tmp_result_list = []
#         code_list = []
#         for i_code in X_val_df[(X_val_df.Date == i)].SecuritiesCode.unique():
#             temp_df = X_val_df[(X_val_df.Date == i)&(X_val_df.SecuritiesCode == i_code)].drop(['Date','SecuritiesCode'],axis=1)
#             tmp_result = models[code].predict(temp_df[columns])
#             tmp_result_list.append(tmp_result)
#             code_list.append(code_list)
#         temp_df = pd.DataFrame({"pred":tmp_result_list, })
#         temp_df["Rank"] = (temp_df["pred"].rank(method="first", ascending=False)-1).astype(int)
#         rank.append(temp_df["Rank"].values)

#     stock_rank=pd.Series([x for y in rank for x in y], name="Rank")
#     df_tmp=pd.concat([X_val_df.reset_index(drop=True),stock_rank,
#                   prices[prices.Date.isin(val_dates)]['Target'].reset_index(drop=True)], axis=1)
#     sharpe=calc_spread_return_sharpe(df_tmp)
    sharpe_ratio.append(sharpe)
    print("Valid Sharpe: {}".format(sharpe))
    
    
#     del X_train, y_train,  X_val, y_val
#     gc.collect()
    
print("\nAverage cross-validation Sharpe Ratio: {:.4f}, standard deviation = {:.2f}.".format(np.mean(sharpe_ratio),np.std(sharpe_ratio)))

# （Predict）

In [ ]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()    

In [ ]:
data = df.copy()
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    display(prices)
    prices["Date"] = pd.to_datetime(prices["Date"])
#     data = prices.drop_duplicates(["SecuritiesCode", "Date"], keep="last").sort_values(["SecuritiesCode", "Date"]).reset_index(drop=True)
    data = data.append(prices).drop_duplicates(["SecuritiesCode", "Date"], keep="last").sort_values(["SecuritiesCode", "Date"]).reset_index(drop=True)
    data = make_features(data)
    
    d = sample_prediction[["Date","SecuritiesCode"]].reset_index()
    d["Date"] = pd.to_datetime(d["Date"])
    d = d.merge(data, on=["Date","SecuritiesCode"])
    for code, _d in d.groupby("SecuritiesCode"):
        d.loc[_d.index, "Pred"] = models[code].predict(_d[columns])#予測
    d = d.sort_values(by="Pred", ascending=False).set_index("index")
    d["Rank"] = np.arange(0,2000)
    
    d = d.sort_index()
    sample_prediction["Rank"] = d["Rank"]
    
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    env.predict(submission)